# IMPORTING LIBRARIES

In [1]:
import pandas as pd
import numpy as np
import imblearn
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn import linear_model
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report, confusion_matrix

# READING CSV FILE

In [2]:
data = pd.read_csv("./CC.csv")
data.head()

,Unnamed: 0,Time,V1,V2,V3,V4,V5,V6,V7,V8,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,1,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,2,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,3,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,4,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,5,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [3]:
# now let us check in the number of Percentage
Count_Normal_transacation = len(data[data["Class"]==0]) # normal transaction are repersented by 0
Count_Fraud_transacation = len(data[data["Class"]==1]) # fraud by 1
Percentage_of_Normal_transacation = Count_Normal_transacation/(Count_Normal_transacation+Count_Fraud_transacation)
print("percentage of normal transacation is",Percentage_of_Normal_transacation*100)
Percentage_of_Fraud_transacation= Count_Fraud_transacation/(Count_Normal_transacation+Count_Fraud_transacation)
print("percentage of fraud transacation",Percentage_of_Fraud_transacation*100)


percentage of normal transacation is 99.82725143693798
percentage of fraud transacation 0.1727485630620034


In [4]:
print("length of training data",len(data))
print("length of normal data",len(data[data["Class"]==0]))
print("length of fraud  data",len(data[data["Class"]==1]))

length of training data 284807
length of normal data 284315
length of fraud  data 492


In [5]:
X = data[['V1','V2','V3','V4','V5','V6','V7','V8','V9','V10','V11','V12','V13','V14','V15','V16','V17','V18','V19',
          'V20','V21','V22','V23','V24','V25','V26','V27','V28','Amount']]
Y = data['Class']

# Oversampling the data

In [6]:
from imblearn.over_sampling import SMOTE

os = SMOTE(random_state=0) 
columns = X.columns

os_data_X,os_data_y=os.fit_sample(X,Y)
os_data_X = pd.DataFrame(data=os_data_X,columns=columns )
os_data_y= pd.DataFrame(data=os_data_y,columns=["Class"])
# we can Check the numbers of our data
print("length of oversampled data is ",len(os_data_X))
print("Number of normal transcation in oversampled data",len(os_data_y[os_data_y["Class"]==0]))
print("No.of fraud transcation",len(os_data_y[os_data_y["Class"]==1]))
print("Proportion of Normal data in oversampled data is ",len(os_data_y[os_data_y["Class"]==0])/len(os_data_X))
print("Proportion of fraud data in oversampled data is ",len(os_data_y[os_data_y["Class"]==1])/len(os_data_X))

length of oversampled data is  568630
Number of normal transcation in oversampled data 284315
No.of fraud transcation 284315
Proportion of Normal data in oversampled data is  0.5
Proportion of fraud data in oversampled data is  0.5


In [7]:
os_data_X

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.620000
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.690000
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.660000
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.500000
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.990000
5,-0.425966,0.960523,1.141109,-0.168252,0.420987,-0.029728,0.476201,0.260314,-0.568671,-0.371407,...,0.084968,-0.208254,-0.559825,-0.026398,-0.371427,-0.232794,0.105915,0.253844,0.081080,3.670000
6,1.229658,0.141004,0.045371,1.202613,0.191881,0.272708,-0.005159,0.081213,0.464960,-0.099254,...,-0.219633,-0.167716,-0.270710,-0.154104,-0.780055,0.750137,-0.257237,0.034507,0.005168,4.990000
7,-0.644269,1.417964,1.074380,-0.492199,0.948934,0.428118,1.120631,-3.807864,0.615375,1.249376,...,-0.156742,1.943465,-1.015455,0.057504,-0.649709,-0.415267,-0.051634,-1.206921,-1.085339,40.800000
8,-0.894286,0.286157,-0.113192,-0.271526,2.669599,3.721818,0.370145,0.851084,-0.392048,-0.410430,...,0.052736,-0.073425,-0.268092,-0.204233,1.011592,0.373205,-0.384157,0.011747,0.142404,93.200000
9,-0.338262,1.119593,1.044367,-0.222187,0.499361,-0.246761,0.651583,0.069539,-0.736727,-0.366846,...,0.203711,-0.246914,-0.633753,-0.120794,-0.385050,-0.069733,0.094199,0.246219,0.083076,3.680000


# SPLITTING THE DATASET INTO TRAIN AND TEST

In [8]:
X_over = os_data_X[['V1','V2','V3','V4','V5','V6','V7','V8','V9','V10','V11','V12','V13','V14','V15','V16','V17','V18','V19',
               'V20','V21','V22','V23','V24','V25','V26','V27','V28','Amount']]
y_over = os_data_y['Class']

In [9]:
y_over

0         0
1         0
2         0
3         0
4         0
5         0
6         0
7         0
8         0
9         0
10        0
11        0
12        0
13        0
14        0
15        0
16        0
17        0
18        0
19        0
20        0
21        0
22        0
23        0
24        0
25        0
26        0
27        0
28        0
29        0
         ..
568600    1
568601    1
568602    1
568603    1
568604    1
568605    1
568606    1
568607    1
568608    1
568609    1
568610    1
568611    1
568612    1
568613    1
568614    1
568615    1
568616    1
568617    1
568618    1
568619    1
568620    1
568621    1
568622    1
568623    1
568624    1
568625    1
568626    1
568627    1
568628    1
568629    1
Name: Class, Length: 568630, dtype: int64

In [10]:

X_over_train, X_over_test, y_over_train, y_over_test = train_test_split(X_over,y_over,test_size = 0.3, random_state = 0)

# RFE ON SVM

In [ ]:
from sklearn.svm import SVC  

model = SVC(kernel='linear')
rfe = RFE(model)
fit = rfe.fit(X_over_train,y_over_train)


In [ ]:
print("Number of Features: %d"% fit.n_features_)
print("Selected features: %s"% fit.support_)
print("Ranking of features: %s"% fit.ranking_)  

# MODEL FITTING ON TRAIN DATA

In [14]:
X = X_over_train[['V1','V2','V3','V4','V5','V8','V10','V11','V12','V14','V16','V17','V18','Amount']]

In [17]:
model.fit(X, y_over_train)  
y_pred = model.predict(X)  
print(confusion_matrix(y_over_train,y_pred))  
print(classification_report(y_over_train,y_pred)) 
print(accuracy_score(y_over_train, y_pred))

[[199137      6]
 [     0 198898]]
             precision    recall  f1-score   support

          0       1.00      1.00      1.00    199143
          1       1.00      1.00      1.00    198898

avg / total       1.00      1.00      1.00    398041

0.9999849261759467


## Test data accuracy

In [18]:
# regressor = RandomForestClassifier(n_estimators=20, random_state=0)  
X2 = X_over_test[['V1','V2','V3','V4','V5','V8','V10','V11','V12','V14','V16','V17','V18','Amount']]
# regressor.fit(X2, y_under_test)  
y_pred = model.predict(X2)  
print(confusion_matrix(y_over_test,y_pred))  
print(classification_report(y_over_test,y_pred))  
print(accuracy_score(y_over_test, y_pred))  

[[85147    25]
 [   13 85404]]
             precision    recall  f1-score   support

          0       1.00      1.00      1.00     85172
          1       1.00      1.00      1.00     85417

avg / total       1.00      1.00      1.00    170589

0.999777242377879


## Grid search 

In [35]:
from sklearn.grid_search import GridSearchCV
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
# Build a classification task using 3 informative features
X, y_over_train = make_classification(n_samples=50,
                                      n_features=10,
                                      n_informative=3,
                                      n_redundant=0,
                                      n_repeated=0,
                                      n_classes=2,
                                      random_state=0,
                                      shuffle=False)


rfc = RandomForestClassifier(n_jobs=-1,max_features= 'sqrt' ,n_estimators=50, oob_score = True) 

param_grid = { 
    'n_estimators': [200, 700],
    'max_features': ['auto', 'sqrt', 'log2']
}

CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
CV_rfc.fit(X, y_over_train)
print (CV_rfc.best_params_)

C:\Users\Komal\AppData\Roaming\Python\Python37\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


{'max_features': 'sqrt', 'n_estimators': 200}


In [36]:
CV_rfc.score(X, y_over_train)

1.0

In [37]:

CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
CV_rfc.fit(X2, y_over_test)
print (CV_rfc.best_params_)

{'max_features': 'auto', 'n_estimators': 200}


In [38]:
CV_rfc.score(X2, y_over_test)

1.0

In [58]:
y_over_train


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1])

In [56]:
X

,V3,V4,V9,V10,V11,V12,V14,V16,V17
212637,-0.867900,-0.534643,-0.181912,-1.056287,-1.050413,-0.090738,-0.686628,0.522578,0.464592
535831,-2.010618,4.228306,-1.234631,-2.424904,2.713344,-5.654222,-6.797630,-2.418108,-3.575750
182914,0.335179,-0.246120,-0.527933,-0.723643,-1.934401,-1.112214,0.603295,-0.320949,-0.487517
402648,-4.772336,3.235866,-0.823270,-5.377625,5.524205,-7.234969,-6.396275,-3.133312,-4.832563
80440,1.157308,-0.183143,-0.685132,-0.395156,1.470742,0.761061,-0.289177,0.366603,-0.101840
47452,0.611499,-0.516958,0.110167,0.262034,-1.371733,-0.217701,-1.008055,0.725110,0.594513
225669,-3.582175,1.337566,1.133534,-0.307708,1.259998,0.441839,-0.535109,1.114926,-0.436689
190802,-0.382970,-0.739292,-1.621996,0.325129,-0.209041,0.083971,0.763928,-1.286721,-0.299431
138519,0.054452,-3.433765,-0.407771,-1.169569,0.866920,1.487880,0.399744,-1.839039,-0.223760
259226,-0.510240,-0.439358,0.245697,0.823936,0.102827,0.200951,0.100684,-1.028763,-0.596802


# K BEST ON Random Forest

In [82]:
#array = under_sample.values
test = SelectKBest(score_func=f_classif, k=10)
fit = test.fit(X_over_train, y_over_train)
print("scores_:",test.scores_)
print("pvalues_:",test.pvalues_)
print("selected index:",test.get_support(True))
print("after transform:",test.transform(X_over_train)) 

scores_: [8.90094295e+04 1.34490388e+05 1.88238248e+05 4.20919510e+05
 7.01624088e+04 8.27595776e+04 1.17168375e+05 2.23421604e+03
 2.02211994e+05 2.80165278e+05 3.89901322e+05 3.69625988e+05
 1.92694387e+03 5.77083297e+05 5.09794479e+02 2.29924455e+05
 1.94388512e+05 1.19822984e+05 3.01968294e+04 1.46847404e+04
 7.76561530e+03 1.42699446e+01 6.32743066e+01 4.76825994e+03
 1.19644623e+03 1.45353018e+03 4.46225074e+03 4.20967142e+03
 1.73454612e+03]
pvalues_: [0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
 0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
 0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
 0.00000000e+000 0.00000000e+000 8.28014570e-113 0.00000000e+000
 0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
 0.00000000e+000 1.58396646e-004 1.80301758e-015 0.00000000e+000
 8.87098339e-262 1.84523094e-317 0.00000000e+000 0.00000000e+000
 0.00000000e+000]
selected index: [ 1  2  3  8  9 10 11 13 15 16]
after transform:

# MODEL FITTING ON TRAIN DATA

## Training Accuracy

In [83]:
model = RandomForestClassifier()
X = X_over_train[['V2','V3','V4','V9','V10','V11','V12','V14','V16','V17']]
model.fit(X, y_over_train)
y_pred = model.predict(X)
print(confusion_matrix(y_over_train,y_pred))  
print(classification_report(y_over_train,y_pred))  
print(accuracy_score(y_over_train, y_pred))


[[199141      2]
 [     3 198895]]
             precision    recall  f1-score   support

          0       1.00      1.00      1.00    199143
          1       1.00      1.00      1.00    198898

avg / total       1.00      1.00      1.00    398041

0.9999874384799556


# MODEL FITTING ON TEST DATA

In [86]:
## fitiing the model
X2 = X_over_test[['V2','V3','V4','V9','V10','V11','V12','V14','V16','V17']]
# logreg.fit(X2, y_under_test)

In [87]:

classifier = LogisticRegression(random_state = 0)
classifier.fit(X2, y_over_test)
y_pred = classifier.predict(X2)
print(confusion_matrix(y_over_test,y_pred))  
print(classification_report(y_over_test,y_pred)) 
print(accuracy_score(y_over_test, y_pred))


[[83454  1718]
 [ 6623 78794]]
             precision    recall  f1-score   support

          0       0.93      0.98      0.95     85172
          1       0.98      0.92      0.95     85417

avg / total       0.95      0.95      0.95    170589

0.9511047019444395


## Grid Search

In [88]:
from sklearn.grid_search import GridSearchCV
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
# Build a classification task using 3 informative features
X, y_over_train = make_classification(n_samples=50,
                                      n_features=10,
                                      n_informative=3,
                                      n_redundant=0,
                                      n_repeated=0,
                                      n_classes=2,
                                      random_state=0,
                                      shuffle=False)


rfc = RandomForestClassifier(n_jobs=-1,max_features= 'sqrt' ,n_estimators=50, oob_score = True) 

param_grid = { 
    'n_estimators': [200, 700],
    'max_features': ['auto', 'sqrt', 'log2']
}

CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
CV_rfc.fit(X, y_over_train)
print (CV_rfc.best_params_)

{'max_features': 'sqrt', 'n_estimators': 200}


In [89]:
CV_rfc.score(X, y_over_train)

1.0

In [ ]:
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
CV_rfc.fit(X2, y_over_test)
print (CV_rfc.best_params_)

In [ ]:
CV_rfc.score(X2, y_over_test)

# PCA ON LOGISTIC REGRESSION

In [ ]:
pca = PCA(n_components=5)
X_over_train = pca.fit_transform(X_over_train)
X_over_test = pca.transform(X_over_test)
explained_variance = pca.explained_variance_ratio_
print(explained_variance)

# MODEL FITTING ON TRAIN & TEST DATA

In [59]:
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_under_train, y_under_train)
y_pred = classifier.predict(X_under_test)
print(confusion_matrix(y_under_test,y_pred))  
print(classification_report(y_under_test,y_pred)) 
print(accuracy_score(y_under_test, y_pred))

[[147   2]
 [ 16 131]]
             precision    recall  f1-score   support

          0       0.90      0.99      0.94       149
          1       0.98      0.89      0.94       147

avg / total       0.94      0.94      0.94       296

0.9391891891891891
